<a href="https://colab.research.google.com/github/RockhoRockho/BFS-samsung-Question/blob/main/17_Rolling_Average.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from glob import glob
# 프로그램이 의도한 상황으로 돌아가고 있는 중인지 상태 확인 모듈
from tqdm import tqdm

In [ ]:
LABEL_INT_DICT = np.unique(pd.read_csv('drive/MyDrive/ucf11_train_vid.csv')['label'])
pprint(LABEL_INT_DICT)
LABEL_STR_DICT = {k:v for v,k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_STR_DICT)

In [ ]:
import cv2
import os
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
model = keras.models.load_model(
    'drive/MyDrive/ucf11_weights/10-0.98.hdf5',
    custom_objects={'KerasLayer': hub.KerasLayer}
)

model.build([None, 256, 256, 3])
model.summary()

In [ ]:
import pandas as pd
import numpy as np

csv_path = 'drive/MyDrive/ucf11_valid_vid.csv'
valid_df = pd.read_csv(csv_path)

idx = 21
elem = valid_df.iloc[idx]

cap = cv2.VideoCapture(
    elem['file_path']
)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter('ori.avi', fourcc, 30, (256, 256), True)

while True:
  ret, frame = cap.read()
  if not ret:
    break

  frame_ = frame.copy()
  frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
  frame_ = cv2.resize(frame_, (256, 256))
  frame_ = frame_.astype('float32')
  frame_ = frame_ / 255.

  pred = model.predict(frame_[np.newaxis, ...])
  label = LABEL_INT_DICT[np.argmax(pred)]

  frame = cv2.resize(frame, (256, 256))
  cv2.putText(frame, label, (50, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 5)
  writer.write(frame)
  
cap.release()
writer.release()

In [ ]:
cap = cv2.VideoCapture(
    elem['file_path']
)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter('roll.avi', fourcc, 30, (256, 256), True)

queue = []
while True:
  ret, frame = cap.read()
  if not ret:
    break

  frame_ = frame.copy()
  frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
  frame_ = cv2.resize(frame_, (256, 256))
  frame_ = frame_.astype('float32')
  frame_ = frame_ / 255.

  pred = model.predict(frame_[np.newaxis, ...])
  queue.append(pred)
  
  results = np.array(queue).mean(axis=0)
  label = LABEL_INT_DICT[np.argmax(results)]

  frame = cv2.resize(frame, (256, 256))
  cv2.putText(frame, label, (50, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 5)
  writer.write(frame)
  
cap.release()
writer.release()